In [ ]:
import os.path as path
from datetime import datetime
import script.utility as util

In [ ]:
inertial_log_file = "foo_inertial_aggmr.pkl"
topcon_log_file = "foo.txt"

inertial_ts, inertial_val = util.load_inertial_log(inertial_log_file)
topcon_ts, topcon_pos, topcon_height = util.make_ts_unique(*util.load_topcon_log(topcon_log_file))
topcon_pos[:, 0] *= -1

find jump action

In [ ]:
min_jump_interval = 0

inertial_jump_idxes = util.find_jump_in_inertial(inertial_ts, inertial_val[:, :3], min_jump_interval)

In [ ]:
min_jump_interval = 0
topcon_begin = None
topcon_end = None
# topcon_begin = datetime(2000, 1, 1, 0, 0)
# topcon_end = datetime(2000, 1, 1, 0, 5)

topcon_jump_idxes = util.find_jump_in_topcon(topcon_ts, topcon_pos, topcon_height, min_jump_interval, topcon_begin, topcon_end)

synchronize logs

In [ ]:
padding = 100
use_jump_idxes = "both"

cut_inertial_ts = util.cut_with_padding(inertial_ts, inertial_jump_idxes, padding)
cut_inertial_val = util.cut_with_padding(inertial_val, inertial_jump_idxes, padding)
resampled_topcon_pos, resampled_topcon_height = util.adjust_freq(cut_inertial_ts, util.adjust_ts_offset(inertial_jump_idxes, inertial_ts, topcon_jump_idxes, topcon_ts, use_jump_idxes), topcon_pos, topcon_height)

correct start point and direction

In [ ]:
ref_direct_tan = 2
rot_angle = 0

corrected_topcon_pos = util.rot(rot_angle, resampled_topcon_pos - resampled_topcon_pos[0])
util.vis_tj(corrected_topcon_pos, ref_direct_tan)

split data

In [ ]:
min_split_interval = 0
split_factor = "y_min"

split_data = util.split(cut_inertial_ts, cut_inertial_val, corrected_topcon_pos, resampled_topcon_height, split_factor, min_split_interval)
util.plot(split_data)

export synchronized log

In [ ]:
dir_name = path.splitext(path.basename(inertial_log_file))[0]

util.export(split_data, dir_name)
util.write_conf(dir_name, padding, rot_angle)